

# Nettoyage de la base de données

### <u>1. Description de la base de données

**Les données** </u>

La base de données utilisée est celle de Data.gouv qui liste les festivals en France. Les critères de sélection des festivals sont les suivants :
- avoir eu lieu en 2019
- avoir connu au moins deux éditions en 2019
- se dérouler sur plus d’une journée
- compter au moins 5 spectacles, représentations, concerts ou projections
<u> 



**Description de la base de données** </u>

La base de donnée regroupe les éléments suivants :
- Identité du festival : nom, disciplines dominantes, sous-catégorie au sein des disciplines
- Données géographiques : région de déroulement, département, adresse postale, géocodage xy (coordonnées)
- Données administratives : identifiant, code insee...
- Données temporelles : période de déroulement, date de création...
- Trouver/communiquer avec le festival : site internet et adresse mail.

### <u>2. Suppression des données inutiles 
</u>

On a choisit de supprimer les colonnes suivantes : 
- Code postal (de la commune principale de déroulement)
- Code Insee commune
- Code Insee EPCI
- Libellé EPCI	
- Numéro de voie	 
- Type de voie (rue, Avenue, boulevard, etc.)	
- Nom de la voie	
- Adresse postale	
- Complément d'adresse (facultatif)
- Décennie de création du festival
- Année de création du festival
- Identifiant Agence A
- identifiant CNM

On a aussi fait le choix de ne conserver que les festivals se déroulant en France métropolitaine. 


In [113]:
import pandas as pd


# Charger le fichier CSV
df = pd.read_csv('festivals_en_France.csv', sep=';', encoding='utf-8-sig')

# Afficher les premières lignes pour vérifier le contenu
print(df.head())


# Supprimer les colonnes qui ne nous intéressent pas
df = df.drop(["Code postal (de la commune principale de déroulement)", "Code Insee commune", "Code Insee EPCI", "Libellé EPCI", "Numéro de voie", "Type de voie (rue, Avenue, boulevard, etc.)", "Nom de la voie", "Adresse postale", "Complément d'adresse (facultatif)", "Décennie de création du festival", "Année de création du festival", "Identifiant Agence A", "identifiant CNM"], axis=1)

# Afficher les colonnes restantes pour vérifier
print(df.columns.tolist())


# Supprimer les festivals hors France métropolitaine
regions_non_metropolitaines = ["Guadeloupe", "Martinique", "Guyane", "La Réunion", "Mayotte", "Polynésie française", "Saint-Pierre-et-Miquelon", "Saint-Barthélemy"]
df = df[~df["Région principale de déroulement"].isin(regions_non_metropolitaines)]

# Vérifier le contenu après suppression
print(df["Région principale de déroulement"].unique())


            Nom du festival Envergure territoriale  \
0          Festival andalou              Régionale   
1  Festival Rêves d'enfants                    NaN   
2      Printemps de Paroles                    NaN   
3                 Pharaonic                    NaN   
4           Sarcus Festival                    NaN   

  Région principale de déroulement Département principal de déroulement  \
0       Provence-Alpes-Côte d'Azur                             Vaucluse   
1       Provence-Alpes-Côte d'Azur                         Hautes-Alpes   
2                    Île-de-France                       Seine-et-Marne   
3             Auvergne-Rhône-Alpes                               Savoie   
4              Centre-Val de Loire                       Indre-et-Loire   

  Commune principale de déroulement  \
0                           Avignon   
1                          Briançon   
2                Bussy-Saint-Martin   
3                          Chambéry   
4              Chemillé-sur-I

### <u>3. Modification et mise en forme des données  

**Transformation des données** </u>

Transformation des données sous forme de listes de listes. La première clef utilisée est nom du festival_identifiant car il y a des festivals ayant le même nom mais se déroulant sur des communes différentes. Ajouter l'identifiant permet d'avoir une clef unique. 

On veut quelque chose de cette forme : {"nom-festival_id" : {"nom-festival:..., "date":..., }}, {"nom-festival_id2" : {...} }, etc. 





In [114]:
# Construire un dictionnaire où chaque clé principale est "Nom du festival_Identifiant"
dictionnaire = {}
for _, row in df.iterrows():
    # Utiliser la colonne "Identifiant" du DataFrame comme partie de la clé
    cle_principale = f"{row['Nom du festival']}_{row['Identifiant']}"
    
    # Ajouter un sous-dictionnaire contenant toutes les colonnes, sauf la colonne "Identifiant"
    dictionnaire[cle_principale] = row.drop("Identifiant").to_dict()

# Afficher un exemple
list(dictionnaire.items())[:5]  # Afficher les 5 premières entrées du dictionnaire pour vérifier


[('Festival andalou_FEST_84007_514',
  {'Nom du festival': 'Festival andalou',
   'Envergure territoriale': 'Régionale',
   'Région principale de déroulement': "Provence-Alpes-Côte d'Azur",
   'Département principal de déroulement': 'Vaucluse',
   'Commune principale de déroulement': 'Avignon',
   'Site internet du festival': 'https://lefestivalandalou.com/',
   'Adresse e-mail': 'contact@lefestivalandalou.com',
   'Discipline dominante': 'Musique',
   'Sous-catégorie spectacle vivant': nan,
   'Sous-catégorie musique': 'Musiques du monde',
   'Sous-catégorie Musique CNM': '09- Pluridisciplinaire',
   'Sous-catégorie cinéma et audiovisuel': nan,
   'Sous-catégorie arts visuels et arts numériques': nan,
   'Sous-catégorie livre et littérature': nan,
   'Période principale de déroulement du festival': 'Avant-saison (1er janvier - 20 juin)',
   'Géocodage xy': '43.9352448339, 4.84071572505'}),
 ("Festival Rêves d'enfants_FEST_05023_1069",
  {'Nom du festival': "Festival Rêves d'enfants",


<u>

**La catégorie "pluridisciplinaire"**  </u>

Il s'agit ici d'expliciter la catégorie "pluridisciplinaire". En effet, l'utilisateur va cocher le type de festival qu'il souhaite : il faut donc spécifier à quelles disciplines appartient le festival pour que celui-ci apparaisse lorsque l'utilisateur cochera. Ainsi, si la colonne "Sous-catégorie [...]" n'est pas vide, on renomme la colonne "Discipline dominante" avec le nom de la sous-catégorie.

**Par exemple** : le festival "Rêves d'enfants" est noté pluridisciplinaire et est un festival de littérature et de spectacle vivant. Il faut qu'il apparaisse lorsque l'utilisateur coche "Littérature" ou "Spectacle vivant". On renomme ainsi la colonne "Discipline dominante" "Livre, littérature et Spectacle vivant". 


Il reste néanmoins 290 festivals marqués comme "pluridisciplinaire" mais sans sous-catégorie renseignée. Nous avons donc choisi de supprimer ces festivals pour deux raisons : 
1. Dans le questionnaire, l'utilisateur clique sur une catégorie de festival qu'il souhaite, si le festival n'en a pas il ne peut cliquer dessus. 
2. Il y a trop de festivals dans ce cas (290) pour pouvoir ajouter manuellement les catégories en question. 

**Par exemple**, le festival Printemps de paroles est noté pluridisciplinaire, mais avec aucune sous-catégorie associée.

In [115]:
# Liste des colonnes sous-catégories que l'on analyse
colonnes_sous_categories = [
    "Sous-catégorie spectacle vivant",
    "Sous-catégorie musique",
    "Sous-catégorie cinéma et audiovisuel",
    "Sous-catégorie arts visuels et arts numériques",
    "Sous-catégorie livre et littérature"
]

# Dictionnaire pour associer les colonnes de sous-catégorie à leurs noms dans "Discipline dominante"
mapping_discipline = {
    "Sous-catégorie spectacle vivant": "Spectacle vivant",
    "Sous-catégorie musique": "Musique",
    "Sous-catégorie cinéma et audiovisuel": "Cinéma et audiovisuel",
    "Sous-catégorie arts visuels et arts numériques": "Arts visuels, arts numériques",
    "Sous-catégorie livre et littérature": "Livre et littérature"
}

# Fonction pour renommer la discipline dominante en respectant les correspondances spécifiques
def renommer_discipline(row):
    # Si "Discipline dominante" est pluridisciplinaire
    if row["Discipline dominante"] == "Pluridisciplinaire":
        # Liste des disciplines à partir des sous-catégories non vides
        sous_categories = [mapping_discipline[col] for col in colonnes_sous_categories if pd.notna(row[col])]
        # Si au moins une sous-catégorie est trouvée, les utiliser
        if sous_categories:
            return " et ".join(sous_categories)
        else:
            return "A supprimer"
    # Sinon, garder la valeur d'origine
    return row["Discipline dominante"]

# Appliquer la fonction sur le DataFrame
df["Discipline dominante"] = df.apply(renommer_discipline, axis=1)

# Supprimer les festivals marqués "A supprimer"
df = df[df["Discipline dominante"] != "A supprimer"]

# Vérifier les résultats
print(df[["Nom du festival", "Discipline dominante"]].head(50))


                                      Nom du festival  \
0                                    Festival andalou   
1                            Festival Rêves d'enfants   
3                                           Pharaonic   
4                                     Sarcus Festival   
5                                    Jazz in Fougères   
6                                            L'aparté   
7                    Art contemporain en milieu rural   
8                        Les Musicales du Noirmoutier   
10                                         Ai que Bom   
11                                 Printemps des Rues   
12                               Rhinoférock festival   
13                             Festival Cirq'ô champs   
14                               Métis Plaine Commune   
15                                  Au Foin De La Rue   
16                                    La Dive Musique   
17                             Nuits musicales d'Uzès   
18                             

<u>

**Modification des sous-catégories** </u>

Pour plus de clarté, nous avons choisis de modifier les sous-catégories présentes dans la base de données qui étaient beaucoup trop nombreuses et les avons réduits à 10 par catégorie. 

**La catégorie Spectacle vivant :**

In [116]:
import re  # Importer le module pour gérer les séparateurs multiples

# Dictionnaire élargi

regroupements_spectacle_vivant = {
    "Théâtre": [
        "Théâtre", "Théâtre - humour", "Théâtre ; Lecture publique", "Arts du théâtre",
        "Théâtre, Danse", "Théâtre, Marionnettes", "Théâtre, arts du conte",
        "Théâtre amateur", "Théâtre de rue", "Arts du théâtre théâtre d'humour",
        "Théâtre musical", "Théâtre - humour comédie", "Théâtre forain", "Du théâtre",
        "Contes", "Représentation", "Poésie", "Lecture…", "Arts du théâtre  théâtre d'humour",
        "Piano jazz théâtre"
    ],
    "Danse": [
        "Danse", "Danses traditionnelles", "Danse contemporaine", "Danse, théâtre",
        "Danses de rue", "Danse afro-contemporaine", "Danse de music’ hall",
        "Danse classique", "Danse theme monde arabe", "Danse theme  europe de l'est",
        "Danse et concerts tango", "Hip hop (danse)", "Capoeira", "Danse contemporaine et danse traditionnelle",
        "Danses", "Bal sévillan"
    ],
    "Arts de la Rue": [
        "Arts de la Rue", "Cirque et Arts de la rue", "Arts de la rue ; Théâtre",
        "Spectacles de rue", "Arts de la rue et Cirque", "Spectacle de rue",
        "Spectacle déambulatoire", "Spectacle de rues", "Arts de la rue et du cirque",
        "Street art", "Arts de la rue - concert - conte", "Des arts de la rue", "Rue",
        "Arts d la rue", "Cultures urbaines ", "Arts de la rue Musique", "Spectacles de rue concerts"
    ],
    "Cirque": [
        "Arts de la piste", "Nouveau Cirque", "Arts du cirque", "Cirque",
        "Clown", "Pyrotechnie", "Magie nouvelle", "Cirque traditionnel", "Mime", "Du cirque",
        "Spectacles équestres", "Arts du crique", " arts du clown"
    ],
    "Musique et Chant": [
        "Musique", "Musiques traditionnelles", "Chanson", "Concerts", "Opéra",
        "Musiques", "Musique ancienne", "Musique médiévale", "Musique et lectures",
        "Chansons théâtrales", "Blues (rythm'n'blues)", "Comédie musicale",
        "Jazz et musiques improvisées", "Musique classique", "Musiques arméniennes",
        "Reggae", "Rock", "Variétés", "Flamenco (musique)", "Musiques du monde",
        "Musiques amplifiées ou électroniques", "Soul funk", "Concerts pop electro",
        "Du chant", "Lecture en musique", "Musiques actuelles", "Musiques traditionnelles de provence",
        " amérique d'avant guerre) des spectacles pour enfants", "Cinéma", "(jazz", " musique savante ",
        " musique savante ", " Chant", " Bal sévillan", "Concours de chants", "Stage musique danse",
        "Rencontres autour de la musique", " Chant", " musique savante ", "Art lyrique ", "Hip-hop ", "Rap", " hip-hop ", " hip-hop / rap"
    ],
    "Marionnettes et Théâtre d'objets": [
        "Théâtre d'objet", "Marionnettes", "Théâtre d'objets", "Marionnettes et Théâtre visuel",
        "Arts de la marionnette", "Marionnettes et théâtre d'objet", "Marionnettes et théâtre d'objets-théâtre d'ombres",
        "Marionnettes-théâtre d'objets-théâtre d'ombres", "Art du mime et du geste", "Marionettes",
        "Marionnettes et théâtre d’objet", "Marionnette", "Marionnettes et théâtre d'objets"
    ],
    "Spectacles pour Jeune Public": [
        "Conte", "Jeune Public", "Conte musical", "Arts du conte",
        "Spectacle pour enfants", "Théâtre jeune public", "Concours de poésie",
        "Découverte des Amériques pour enfants", "Lecture publique", "De la poésie", " decouverte des ameriques pour enfants"
    ],
    "Performance et Arts Visuels": [
        "Performance", "Arts visuels", "photo", "vidéo", "Exposition",
        "Cinéma et audiovisuel", "Magie", "Sculptures", "Parité homme-femme",
        "Pyrotechnie", "Rencontres voyageurs", "Audiovisuel-cinéma", "Ciné-concert",
        "Lecture publique", "Découverte du patrimoine de l’Occitanie autrement",
        "Découverte du Japon", "Culture asiatique", "Thème culture franco colombienne", "theme culture franco colombienne", "La création artistique",
        "L´innovation technologique", "Le recyclage des matériaux", "Le partage des compétences techniques et artistiques",
        "Performance - installation", "Arts", "Rencontrer sur le theme de l'usage du faux et démocratie"
    ],
    "Humour et Café-Théâtre": [
        "Humour", "Théâtre d'humour/café-théâtre", "Café-Théâtre",
        "Théâtre d'humour", "Rires et saveurs: soirée dégustation et concert d'exception", " arts du clown"
    ],
    "Pluridisciplinaire": [
        "Pluridisciplinaire", "Pluridisciplinaire culture", "Pluridisciplinaire à dominante spectacle vivant",
        "Spectacle vivant pluridisciplinaire", "SV hors MUA", "Spectacle vivant",
        "Dégustation de vin", "Thème du Moyen Âge", "Découverte des Amériques pour enfants",
        "Terroir", "Performances", "Toutes les disciplines du spectacle vivant",
        "Conférence", "Solidarités", "Approche de la langue et la culture occitane",
        "Ateliers ouvertures vers l'autres", "Animaux", "Thème du voyage", "Sur le thème du voyage", "Theme du voyage et les sciences",
        "Theme amerique", "Le partage des compétences techniques et artistiques et les Droits Culturels dans le contexte géopolitique transfrontalier suivant les méthodologies de l´Écologie Acoustique.","Marché producteur locaux",
        "Découvrir le patrimoine de l’Occitanie autrement", "Autres (voix d'enfants)", "Animations", "Spectacles", "Et autres…",
        "Des bonnes tablées", "Representation", "Cultures urbaines ", "Bodégas", "Cultures urbaines ", " "
    ]
}

# Inverser le dictionnaire pour une recherche efficace
inverse_regroupements = {}
for categorie, sous_cats in regroupements_spectacle_vivant.items():
    for sous_cat in sous_cats:
        inverse_regroupements[sous_cat.lower().strip()] = categorie

# Fonction pour attribuer les sous-catégories
def attribuer_sous_categories(row):
    if "Spectacle vivant" not in str(row.get("Discipline dominante", "")):
        return None  # Si ce n'est pas du spectacle vivant, ne rien faire
    
    sous_categorie = row.get("Sous-catégorie spectacle vivant")
    if pd.isna(sous_categorie):  # Si la sous-catégorie est NaN
        return None
    
    sous_categories = re.split(r"[;,]", str(sous_categorie))  # Diviser en cas de multiples sous-catégories
    nouvelles_categories = set()
    for sous_cat in sous_categories:
        sous_cat_normalise = sous_cat.lower().strip()  # Normaliser : minuscule et sans espaces
        if sous_cat_normalise in inverse_regroupements:
            nouvelles_categories.add(inverse_regroupements[sous_cat_normalise])
        else:
            # Débogage pour les sous-catégories non reconnues
            print(f"Sous-catégorie non reconnue : '{sous_cat}' pour le festival '{row['Nom du festival']}'")
    return list(nouvelles_categories) if nouvelles_categories else None

# Appliquer la fonction uniquement si Discipline dominante contient "Spectacle vivant"
df["Nouvelles sous-catégories spectacle vivant"] = df.apply(attribuer_sous_categories, axis=1)



In [117]:
##                  VÉRIFICATION DES RÉSULTATS 

# Filtrer les lignes où "Discipline dominante" contient "Spectacle vivant"
spectacle_vivant_df = df[df["Discipline dominante"].str.contains("Spectacle vivant", na=False)]
    
# Afficher uniquement les colonnes demandées pour les 250 premières lignes
print(spectacle_vivant_df[["Nom du festival", "Région principale de déroulement", "Discipline dominante", "Nouvelles sous-catégories spectacle vivant"]].head(50))


                                       Nom du festival  \
1                             Festival Rêves d'enfants   
6                                             L'aparté   
11                                  Printemps des Rues   
13                              Festival Cirq'ô champs   
21                                  Théâtre de verdure   
25                        Festival Le Vent sur l'Arbre   
27                                    Tréteaux nomades   
41                                               Imago   
49                                Thoiry 100 Histoires   
50                           Des cirques indisciplinés   
54       Festival de danse Cannes - Côte d'Azur France   
55               Festival de la gastronomie provencale   
58                      Le festival Les AnthropoScènes   
70                                Festival Tête en L’R   
74                                        Les Odyssées   
78                                       Petit Patapon   
88          Fe

In [118]:
# Suppression des festivals pour lesquels aucune sous-catégorie n'est renseignée. - il y en a environ 300 sur les 1600 festivals de Spectacle vivant. 

# Filtrer pour conserver uniquement les festivals pour lesquels la discipline dominante contient "Spectacle vivant"
df = df[
    ~(
        df["Discipline dominante"].str.contains("Spectacle vivant", na=False) &  
        (df["Nouvelles sous-catégories spectacle vivant"].isna() |  # La sous-catégorie est NaN
         (df["Nouvelles sous-catégories spectacle vivant"] == "None"))  # La sous-catégorie est "None"
    )
]

##                  VÉRIFICATION DES RÉSULTATS 

# Afficher les résultats pour vérification
print(df[["Nom du festival", "Sous-catégorie spectacle vivant", "Nouvelles sous-catégories spectacle vivant"]].head(20))

# Filtrer les lignes où "Discipline dominante" contient "Spectacle vivant"
spectacle_vivant_df = df[df["Discipline dominante"].str.contains("Spectacle vivant", na=False)]
print(spectacle_vivant_df[["Nom du festival", "Discipline dominante", "Nouvelles sous-catégories spectacle vivant"]].head(50))


                                      Nom du festival  \
0                                    Festival andalou   
1                            Festival Rêves d'enfants   
3                                           Pharaonic   
4                                     Sarcus Festival   
5                                    Jazz in Fougères   
6                                            L'aparté   
7                    Art contemporain en milieu rural   
8                        Les Musicales du Noirmoutier   
10                                         Ai que Bom   
12                               Rhinoférock festival   
13                             Festival Cirq'ô champs   
14                               Métis Plaine Commune   
15                                  Au Foin De La Rue   
16                                    La Dive Musique   
17                             Nuits musicales d'Uzès   
18                                     Musique à Vars   
20  Festival international de g

**La catégorie Arts visuels, arts numériques :**

In [ ]:
import re

# Dictionnaire des regroupements pour Arts Visuels
regroupements_arts_visuels = {
    "Arts plastiques et visuels": ["Arts visuels", "Arts plastiques", "Arts plastiques et visuels","Arts visuels (arts multimédia)", "Art plastique", "Création contemporaine","Installation", "Installations", "Installations artistiques","Land art", "Parcours Land Art", "Exposition d'art contemporain dans des sites naturels","Arts visuel / Art dans l'espace public / Street Art", "Street photography",
         "Expositions", "Expositions et projections photographie",
        "Exposition à ciel ouvert", "Photo exposition à ciel ouvert",
        "Illustration", "Peinture", "Estampes", "Gravure", "Gravures",
        "Art textile", "Typographie", "Céramique", "Vitrail", "Éditions d'artiste",
        "Autres (estampe)", "Autres (textile)", "Autres (photographie)",
        "Autres : Design", "Art contemporain au sens large", "Art contemporain en général (peinture",
        "Art contemporain et patrimoine", "Expositions d'art contemporain", "Exposition de clocher en clocher", "Sculpture", "Art contemporain", "Arts graphiques"
    ],
    "Arts numériques et vidéo": [
        "Vidéo mapping", "Art vidéo", "Vidéos d'artistes", "Performances multimédias",
        "Programmations croisées avec le cinéma", "Cinéma et audiovisuel",
        "Création sur internet", "Installation numérique", "Oeuvres XR (réalité virtuelle et réalité augmentée)",
        "Vidéo : oeuvres multiples d'artistes", "Micro édition", "Multiples",
        "Livres d'artiste", "Salon d'éditions", "Colloque universitaire", "Arts numériques"
    ],
    "Performance": [
        "Performance", "Performances", "Performance - installation",
        "Performances et spectacles hybrides contemporains entre corps et son",
        "Performances multimédias", "Soirées performances"
    ],
    "Mode et Design": [
        "Mode et design", "Paris Design Week", "Festival international de design - design parade Hyères",
        "Jeune Création", "Street Music"
    ],
    "Métiers d'art": [
        "Métiers d'art", "Autres (artisans d'art)", "Autre : Métier d'art de la Céramique",
        "Calligraphie et Patrimoine Écrit", "Festival des métiers d'art", "Design textile"
    ],
    "Photographie et cinéma": [
        "Photographie", "Photographies", "Expositions de photographie",
        "Projections documentaires", "Photo Montier", "Festival de Street Photography",
        "Films d'artistes", "Cinéma", "Projections"
    ],
    "Arts et sciences": [
        "Arts et sciences visionnaires", "Vulgarisation scientifique",
        "Festival d'idées", "Conférence table ronde thème environnement",
        "Festival artistique et citoyen qui questionne notre rapport à l’autre",
        "Sciences et arts", "Exploration des sciences dans les arts"
    ],
    "Divers": [
        "Divers", "Etc.", "Autres (voix d'enfants)", "Autres (textile)",
        "À l’accueil", "À l’étranger", "Au bien-vivre ensemble dans la diversité", "Pluridisciplinaire"
    ],
    "Street art et espace public": [
        "Street art", "Arts de la rue", "Graffiti", "Art dans l'espace public",
        "Art en plein air", "In situ", "Bien Urbain, art dans (et avec) l'espace public"
    ]
}

regroupements_arts_visuels_numeriques = {
    "Arts numériques": [
        "Numérique", "Arts audiovisuels", "Vidéo", "Video : oeuvres multiples d'artistes",
        "Performance costume", "Performances multimédias", "Installation numérique",
        "Arts visuels numériques", "Création sur internet", "Vulgarisation", "Formes immersives et interactives"
    ],
    "Arts plastiques et visuels": [
        "Arts plastiques", "Sculptures", "Peinture", "Arts plastiques et visuels",
        "Expositions", "Gravure", "Estampes", "Dessin", "Illustration", "Graphisme",
        "Calligraphie et Patrimoine Ecrit", "Art contemporain en général (peinture",
        "Art contemporain au sens large"
    ],
    "Photographie": [
        "Photographie", "Photo", "Photo exposition à ciel ouvert",
        "Photographies", "Street photography", "Vidéo mapping"
    ],
    "Design et architecture": [
        "Design", "Architecture", "Design graphique", "Paysagisme",
        "Mode et design", "Biennale d'architecture et d'urbanisme", 
        "Festival d'architecture", "Festival des Architectures vives"
    ],
    "Arts urbains": [
        "Arts urbains", "Street art", "Graff", "Arts visuels (arts multimédia)",
        "Expériences urbaines"
    ],
    "Performance et multimédia": [
        "Performance", "Performances", "Installation", "Performances multimédias",
        "Ateliers avec les habitants de la ville", "Performances et spectacles hybrides contemporains entre corps et son",
        "Démonstrations", "Rencontres", "Animations diverses", "Performance costume"
    ],
    "Musique et arts sonores": [
        "Musique", "Art sonore et nouvelle musique", "Musiques actuelles",
        "Vinyls", "Nouvelle musique"
    ],
    "Littérature et illustration": [
        "Dessin de presse", "Bande dessinée", "Autre (Dessin)",
        "Calligraphie"
    ],
    "Arts de l'image et audiovisuel": [
        "Audiovisuel", "Documentaire", "Vidéos d'artistes", "Programmations croisées avec le cinéma",
        "Cinéma", "Festival d'idées", "Expositions et projections photographie"
    ],
    "Pluridisciplinaire et autres": [
        "Spectacle vivant", "Culture juive", "Festival artistique et citoyen",
        "Autres : Performances", "Etc.", "Autres (artisans d'art)"
    ]
}



# Inverser le dictionnaire pour recherche
inverse_regroupements_arts_visuels = {}
for categorie, sous_cats in regroupements_arts_visuels.items():
    for sous_cat in sous_cats:
        inverse_regroupements_arts_visuels[sous_cat.lower().strip()] = categorie

# Fonction pour attribuer les sous-catégories

def attribuer_sous_categories_arts_visuels(row):
    discipline_dominante = str(row.get("Discipline dominante", "")).strip().lower()  # Normalisation
    
    # Vérifier si la discipline dominante contient "arts visuels, arts numériques"
    if "arts visuels, arts numériques" not in discipline_dominante:
        return None

    sous_categorie = row.get("Sous-catégorie arts visuels et arts numériques")
    if pd.isna(sous_categorie):  # Si NaN, retourner None
        return None

    sous_categories = re.split(r"[;,]", str(sous_categorie))  # Diviser les sous-catégories multiples
    nouvelles_categories = set()
    for sous_cat in sous_categories:
        sous_cat_normalise = sous_cat.lower().strip()  # Normaliser
        if sous_cat_normalise in inverse_regroupements_arts_visuels:
            nouvelles_categories.add(inverse_regroupements_arts_visuels[sous_cat_normalise])
        else:
            # Afficher les sous-catégories non reconnues
            print(f"Sous-catégorie non reconnue : '{sous_cat.strip()}' pour le festival '{row['Nom du festival']}'")
    return list(nouvelles_categories) if nouvelles_categories else None

# Appliquer la fonction
df["Nouvelles sous-catégories arts visuels"] = df.apply(attribuer_sous_categories_arts_visuels, axis=1)

Sous-catégorie non reconnue : 'Autre : Performances' pour le festival 'Setu'
Sous-catégorie non reconnue : '...)' pour le festival 'Base'art'
Sous-catégorie non reconnue : 'bande dessinée' pour le festival 'Central Vapeur'
Sous-catégorie non reconnue : 'dessin' pour le festival 'Central Vapeur'
Sous-catégorie non reconnue : 'graphisme' pour le festival 'Central Vapeur'
Sous-catégorie non reconnue : 'Photo' pour le festival 'Paris Photo'
Sous-catégorie non reconnue : 'Autre (Dessin)' pour le festival 'Dessin contemporain & populaire'
Sous-catégorie non reconnue : 'musiques actuelles' pour le festival 'Voies off'
Sous-catégorie non reconnue : 'performance costume' pour le festival 'Manga'p'
Sous-catégorie non reconnue : 'démonstrations' pour le festival 'Festival de la sculpture de l’espace Bourdelle'
Sous-catégorie non reconnue : 'rencontres' pour le festival 'Festival de la sculpture de l’espace Bourdelle'
Sous-catégorie non reconnue : 'animations diverses' pour le festival 'Festival d

In [ ]:
##                  VÉRIFICATION DES RÉSULTATS 

# Filtrer les festivals pour lesquels la discipline dominante est "Arts visuels, Arts numériques"
arts_visuels_df = df[df["Discipline dominante"].str.contains("Arts visuels", na=False, case=False)]

# Afficher les premières lignes pour vérifier
print(arts_visuels_df[["Nom du festival", "Discipline dominante", "Nouvelles sous-catégories arts visuels"]].head(50))
